# Import

In [1]:
import numpy as np
import pandas as pd
import cv2 as cv
import cv2 as cv2
from matplotlib import pyplot as plt
import math
import os
import shutil
from pathlib import Path
import glob
import itertools
from itertools import cycle
from scipy import interp

from statistics import mean
from statistics import stdev

!pip install pyradiomics
import radiomics
import SimpleITK as sitk
import six
from skimage.feature import greycomatrix, greycoprops
from array import *
from scipy.cluster.vq import whiten
from scipy.cluster.vq import kmeans

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import label_binarize
from sklearn.decomposition import PCA

from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

import matplotlib.pyplot as plt
import matplotlib.image as img

     |████████████████████████████████| 188 kB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 109 kB 47.7 MB/s eta 0:00:01
     |████████████████████████████████| 500 kB 75.5 MB/s eta 0:00:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=9754d9ea81ea8c32300944daa183808314b4f8ecc2945f9e61cb4a6728edb87a
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


# Folders and Files Paths

In [2]:
# Remember to include '/' at the end of the folder path only

# Folder path for original images
oriBS = '../input/tomato-leaves-original-dataset/Dataset/Bacterial_Spot/'
oriEB = '../input/tomato-leaves-original-dataset/Dataset/Early_Blight/'
oriHL = '../input/tomato-leaves-original-dataset/Dataset/Healthy/'
oriLB = '../input/tomato-leaves-original-dataset/Dataset/Late_Blight/'
oriSLS = '../input/tomato-leaves-original-dataset/Dataset/Septorial_leaf_spot/'
oriYCV = '../input/tomato-leaves-original-dataset/Dataset/Yellow_Curl_Virus/'

# Folder path for segmented images
segBS = './Bacterial_Spot_Segmented/'
segEB = './Early_Blight_Segmented/'
segHL = './Healthy_Segmented/'
segLB = './Late_Blight_Segmented/'
segSLS = './Septorial_Leaf_Spot_Segmented/'
segYCV = './Yellow_Curl_Virus_Segmented/'


# File path for extracted features
feaBS = './Bacterial_Spot_Features.csv'
feaEB = './Early_Blight_Features.csv'
feaHL = './Healthy_Features.csv'
feaLB = './Late_Blight_Features.csv'
feaSLS = './Septorial_Leaf_Spot_Features.csv'
feaYCV = './Yellow_Curl_Virus_Features.csv'

In [3]:
# Create the folders to store segmented images
# If the folders had been created, do not run this cell
os.makedirs(segBS)
os.makedirs(segEB)
os.makedirs(segHL)
os.makedirs(segLB)
os.makedirs(segSLS)
os.makedirs(segYCV)

# ROI Segmentation and Image Pre-processing

In [4]:

def segmentation(originalImageFolder, newFolderPath, cannyMin, cannyMax, numOfControusThreshold, lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro):

    fileList = []
    fileIDList = []

    fileIDList = next(os.walk(originalImageFolder), (None, None, []))[2] 
        
    print('Number of Images: ', len(fileIDList))
    i = 0
    for file in fileIDList:
        imgOriginal = cv.imread(originalImageFolder+file,1)
        imgGray = cv.imread(originalImageFolder+file,0)
        
        
        # Apply Gaussian Blur
        imgGray = cv.GaussianBlur(imgGray,(3,3),0)

        
        # Apply Canny edge
        edges = cv.Canny(imgGray,cannyMin,cannyMax)


        # Apply filter contour by length
        contours, hierarchy = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
        contours = sorted(contours, key=lambda x: cv2.arcLength(x, True))
        if len(contours) > numOfControusThreshold:
            imgContours = np.zeros(imgGray.shape, dtype=np.uint8)
            lowerBound = max(numOfControusThreshold, int(len(contours)/lowerBoundDenominator))
            for j in reversed(range(len(contours) - lowerBound,len(contours))):
                length = cv2.arcLength(contours[j], True)
                cv2.drawContours(imgContours, contours, j, (255), -1)
        else:
            imgContours = edges

            
        # Apply dilation
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernelDil,kernelDil))
        dilation = cv2.dilate(imgContours,kernel,iterations = dilationIt)


        # Find the max contours
        contours, hierarchy = cv2.findContours(dilation,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
        maxContour = max(contours, key = len)


        # Draw it on image
        img = np.zeros( imgGray.shape, dtype=np.uint8)
        img = cv2.drawContours(img, [maxContour], -1, (255),-1)


        # Erosion after filling contour
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernelEro,kernelEro))
        img = cv2.erode(img,kernel,iterations = erodeIt)
        segmented = cv2.bitwise_and(imgOriginal,imgOriginal,mask = img)

        
        # Convert from RGB to LAB colour space
        lab = cv2.cvtColor(segmented, cv2.COLOR_BGR2LAB)
        lab_planes = cv2.split(lab)
        
        
        # Apply CLAHE
        clahe = cv2.createCLAHE(clipLimit=5.0)
        lab_planes[0] = clahe.apply(lab_planes[0])
        lab = cv2.merge(lab_planes)

        
        # Convert back from RGB to LAB colour space
        bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

        
        # Write image into the folder
        cv2.imwrite(newFolderPath + fileIDList[i], bgr)
        i+=1
        
    # Create a zip file fir the folder to be downloaded
    shutil.make_archive(newFolderPath, 'zip', newFolderPath)

In [5]:


cannyMin = 40
cannyMax = 125
numOfControusThreshold = 70
lowerBoundDenominator = 2
dilationIt = 20
erodeIt = 15
kernelDil = 3
kernelEro = 3
segmentation(oriBS, segBS, cannyMin, cannyMax, numOfControusThreshold, \
             lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro)


cannyMin = 80
cannyMax = 125
numOfControusThreshold = 50
lowerBoundDenominator = 2
dilationIt = 20
erodeIt = 15
segmentation(oriEB, segEB, cannyMin, cannyMax, numOfControusThreshold, \
             lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro)


cannyMin = 80
cannyMax = 125
numOfControusThreshold = 30
lowerBoundDenominator = 2
dilationIt = 20
erodeIt = 15

segmentation(oriHL, segHL, cannyMin, cannyMax, numOfControusThreshold, \
             lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro)


cannyMin = 60
cannyMax = 125
numOfControusThreshold = 70
lowerBoundDenominator = 2
dilationIt = 20
erodeIt = 15

segmentation(oriLB, segLB, cannyMin, cannyMax, numOfControusThreshold, \
             lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro)


cannyMin = 100
cannyMax = 125
numOfControusThreshold = 100
lowerBoundDenominator = 4
dilationIt = 30
erodeIt = 20

segmentation(oriSLS, segSLS, cannyMin, cannyMax, numOfControusThreshold, \
             lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro)


cannyMin = 40
cannyMax = 125
numOfControusThreshold = 40
lowerBoundDenominator = 4
dilationIt = 30
erodeIt = 20

segmentation(oriYCV, segYCV, cannyMin, cannyMax, numOfControusThreshold, \
             lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro)

Number of Images:  250
Number of Images:  250
Number of Images:  612
Number of Images:  250
Number of Images:  250
Number of Images:  250


# Feature Extraction

## Colour Histogram

In [6]:

def ColourHistogram(folderPath,filename):
    
    # Read image
    img = cv.imread(folderPath+filename)
    
    # Resize image
    width = int(256)
    height = int(256)
    dim = (width, height)
    img = cv.resize(img, dim, interpolation = cv.INTER_AREA)
    
    # Create df
    x = [i for i in range(0,45)]
    df = pd.DataFrame(columns = x)
    
    # Colour Histogram
    Blue = cv.calcHist([img], [0], None, [256], [0,256])
    Green = cv.calcHist([img], [1], None, [256], [0,256])
    Red = cv.calcHist([img], [2], None, [256], [0,256])

    result_R = [i for i in range(0,15)]
    result_G = [i for i in range(0,15)]
    result_B = [i for i in range(0,15)]

    start = 0
    end = 17
    for i in range(0,15):
        r = np.sum(Red[start:end])
        g = np.sum(Green[start:end])
        b = np.sum(Blue[start:end])
        start = end
        end = end + 17
        result_R[i] = r
        result_G[i] = g
        result_B[i] = b

    finalResult = [filename]
    finalResult = result_R + result_G + result_B + finalResult

    df = df.append([finalResult])
    df['ID'] = df[45]
    df.drop(45, axis = 1, inplace = True)
    df = df.set_index('ID')

    return df

## Dominant Colours

In [7]:

def DominantColour(folderPath,filename):
    
    # Read image
    img = cv.imread(folderPath+filename)
    
    
    # Resize image
    width = int(256)
    height = int(256)
    dim = (width, height)
    img = cv.resize(img, dim, interpolation = cv.INTER_AREA)
    
    
    # Create df
    df = pd.DataFrame()
    
    
    # RGB
    r = []
    g = []
    b = []
    for row in img:
        for temp_r, temp_g, temp_b in row:
            r.append(temp_r)
            g.append(temp_g)
            b.append(temp_b)
            
    dfKMeans = pd.DataFrame({'red' : r,
                          'green' : g,
                          'blue' : b})
 
    dfKMeans['scaled_color_red'] = whiten(dfKMeans['red'])
    dfKMeans['scaled_color_blue'] = whiten(dfKMeans['blue'])
    dfKMeans['scaled_color_green'] = whiten(dfKMeans['green'])

    cluster_centers, _ = kmeans(dfKMeans[['scaled_color_red',
                                    'scaled_color_blue',
                                    'scaled_color_green']], 3)

    dominant_colors = [filename]

    red_std, green_std, blue_std = dfKMeans[['red','green','blue']].std()

    values = [filename]
    for i, cluster_center in enumerate(cluster_centers):
        red_scaled, green_scaled, blue_scaled = cluster_center
        values.append(red_scaled * red_std / 255)
        values.append(green_scaled * green_std / 255)
        values.append(blue_scaled * blue_std / 255)

    while len(values) < 10:
        values.append(pd.NA)
        
    df = df.append([values])
    df.columns = ['ID','Red1','Green1','Blue1','Red2','Green2','Blue2','Red3','Green3','Blue3']
    df = df.set_index('ID')

    return df

## GLCM

In [8]:

def GLCM(folderPath,filename):
    # Read image
    img = cv.imread(folderPath+filename, 0)
    
    # Resize image
    width = int(256)
    height = int(256)
    dim = (width, height)
    img = cv.resize(img, dim, interpolation = cv.INTER_AREA)
    
    df = pd.DataFrame()
    df['ID'] = [filename]

    #Full image
    #GLCM = greycomatrix(img, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])
    GLCM = greycomatrix(img, [1], [0])       
    GLCM_Energy = greycoprops(GLCM, 'energy')[0]
    df['Energy'] = GLCM_Energy
    GLCM_corr = greycoprops(GLCM, 'correlation')[0]
    df['Corr'] = GLCM_corr       
    GLCM_diss = greycoprops(GLCM, 'dissimilarity')[0]
    df['Diss_sim'] = GLCM_diss       
    GLCM_hom = greycoprops(GLCM, 'homogeneity')[0]
    df['Homogen'] = GLCM_hom       
    GLCM_contr = greycoprops(GLCM, 'contrast')[0]
    df['Contrast'] = GLCM_contr

    GLCM2 = greycomatrix(img, [3], [0])       
    GLCM_Energy2 = greycoprops(GLCM2, 'energy')[0]
    df['Energy2'] = GLCM_Energy2
    GLCM_corr2 = greycoprops(GLCM2, 'correlation')[0]
    df['Corr2'] = GLCM_corr2       
    GLCM_diss2 = greycoprops(GLCM2, 'dissimilarity')[0]
    df['Diss_sim2'] = GLCM_diss2       
    GLCM_hom2 = greycoprops(GLCM2, 'homogeneity')[0]
    df['Homogen2'] = GLCM_hom2       
    GLCM_contr2 = greycoprops(GLCM2, 'contrast')[0]
    df['Contrast2'] = GLCM_contr2

    GLCM3 = greycomatrix(img, [5], [0])       
    GLCM_Energy3 = greycoprops(GLCM3, 'energy')[0]
    df['Energy3'] = GLCM_Energy3
    GLCM_corr3 = greycoprops(GLCM3, 'correlation')[0]
    df['Corr3'] = GLCM_corr3       
    GLCM_diss3 = greycoprops(GLCM3, 'dissimilarity')[0]
    df['Diss_sim3'] = GLCM_diss3       
    GLCM_hom3 = greycoprops(GLCM3, 'homogeneity')[0]
    df['Homogen3'] = GLCM_hom3       
    GLCM_contr3 = greycoprops(GLCM3, 'contrast')[0]
    df['Contrast3'] = GLCM_contr3

    GLCM4 = greycomatrix(img, [0], [np.pi/4])       
    GLCM_Energy4 = greycoprops(GLCM4, 'energy')[0]
    df['Energy4'] = GLCM_Energy4
    GLCM_corr4 = greycoprops(GLCM4, 'correlation')[0]
    df['Corr4'] = GLCM_corr4       
    GLCM_diss4 = greycoprops(GLCM4, 'dissimilarity')[0]
    df['Diss_sim4'] = GLCM_diss4       
    GLCM_hom4 = greycoprops(GLCM4, 'homogeneity')[0]
    df['Homogen4'] = GLCM_hom4       
    GLCM_contr4 = greycoprops(GLCM4, 'contrast')[0]
    df['Contrast4'] = GLCM_contr4

    GLCM5 = greycomatrix(img, [0], [np.pi/2])       
    GLCM_Energy5 = greycoprops(GLCM5, 'energy')[0]
    df['Energy5'] = GLCM_Energy5
    GLCM_corr5 = greycoprops(GLCM5, 'correlation')[0]
    df['Corr5'] = GLCM_corr5       
    GLCM_diss5 = greycoprops(GLCM5, 'dissimilarity')[0]
    df['Diss_sim5'] = GLCM_diss5       
    GLCM_hom5 = greycoprops(GLCM5, 'homogeneity')[0]
    df['Homogen5'] = GLCM_hom5       
    GLCM_contr5 = greycoprops(GLCM5, 'contrast')[0]
    df['Contrast5'] = GLCM_contr5

    GLCM6 = greycomatrix(img, [0], [3*np.pi/4])       
    GLCM_Energy6 = greycoprops(GLCM6, 'energy')[0]
    df['Energy6'] = GLCM_Energy6
    GLCM_corr6 = greycoprops(GLCM6, 'correlation')[0]
    df['Corr6'] = GLCM_corr6       
    GLCM_diss6 = greycoprops(GLCM6, 'dissimilarity')[0]
    df['Diss_sim6'] = GLCM_diss6       
    GLCM_hom6 = greycoprops(GLCM6, 'homogeneity')[0]
    df['Homogen6'] = GLCM_hom6       
    GLCM_contr6 = greycoprops(GLCM6, 'contrast')[0]
    df['Contrast6'] = GLCM_contr6

    df = df.set_index('ID')
    return df

## GLSZM

In [9]:

def GLSZM(folderPath, filename):
    # Read image
    #img = cv.imread(filename)
    img = cv.imread(folderPath+filename, 0)
    # Resize image
    width = int(256)
    height = int(256)
    dim = (width, height)
    img = cv.resize(img, dim, interpolation = cv.INTER_AREA)
    
    height = img.shape[0]
    width = img.shape[1]
    tempMask = np.ones((height,width), np.uint8)

    img = sitk.GetImageFromArray(img)
    mask = sitk.GetImageFromArray(tempMask)
    
    # Create df
    df = pd.DataFrame()
    
    # Generate Features
    glszmFeatures = radiomics.glszm.RadiomicsGLSZM(img, mask)
    glszmFeatures.enableAllFeatures()
    result = glszmFeatures.execute()
    
    values = [filename]
    for (key, val) in six.iteritems(result):
        values.append(val.item())

    # Append the value into the df and define the column names
    df = df.append([values])
    df.columns = ['ID', 'GrayLevelNonUniformity', 'GrayLevelNonUniformityNormalized',
       'GrayLevelVariance', 'HighGrayLevelZoneEmphasis', 'LargeAreaEmphasis',
       'LargeAreaHighGrayLevelEmphasis', 'LargeAreaLowGrayLevelEmphasis',
       'LowGrayLevelZoneEmphasis', 'SizeZoneNonUniformity',
       'SizeZoneNonUniformityNormalized', 'SmallAreaEmphasis',
       'SmallAreaHighGrayLevelEmphasis', 'SmallAreaLowGrayLevelEmphasis',
       'ZoneEntropy', 'ZonePercentage', 'ZoneVariance']
    df = df.set_index('ID')
    return df

## Extract All Features

In [10]:

def FeaturesExtractionsAll(folderPath):
    df = pd.DataFrame()
    for fname in sorted(os.listdir(folderPath)):
        colourHist = ColourHistogram(folderPath,fname)
        dominantCol = DominantColour(folderPath,fname)
        glcm = GLCM(folderPath,fname)
        glszm = GLSZM(folderPath,fname)
        dfRow = colourHist.join([dominantCol, glcm, glszm])
        df = df.append(dfRow)
            
    return df

In [ ]:

dfBacterialSpot = FeaturesExtractionsAll(segBS)
dfBacterialSpot.to_csv(feaBS)

dfEarlyBlight = FeaturesExtractionsAll(segEB)
dfEarlyBlight.to_csv(feaEB)

dfHealthy = FeaturesExtractionsAll(segHL)
dfHealthy.to_csv(feaHL)

dfLateBlight = FeaturesExtractionsAll(segLB)
dfLateBlight.to_csv(feaLB)

dfSeptorial = FeaturesExtractionsAll(segSLS)
dfSeptorial.to_csv(feaSLS)

dfYellowCurl = FeaturesExtractionsAll(segYCV)
dfYellowCurl.to_csv(feaYCV)

# Training

## Combine All Classes' Features and Remove Outliers

In [ ]:

def CombineAllDF(folderPaths, labels):
    dfAll = pd.DataFrame()
    i=0
    for paths, label in zip(folderPaths, labels):
        dfs = pd.read_csv(paths)
        
        dfs['Labels'] = labels[i]
        dfs = dfs.set_index('ID')
        
        dfAll = dfAll.append(dfs)
        i=i+1
    dfAll = dfAll.reset_index()
    return dfAll

In [ ]:


# Combine all df from each class into one big df
# So the final df contains a df with all classess, and all features

bacterialSpot = feaBS
earlyBlight = feaEB
healthy = feaHL
lateBlight = feaLB
septorialSpot = feaSLS
yellowCurl = feaYCV


folderPaths = [bacterialSpot,earlyBlight,healthy,lateBlight,septorialSpot,yellowCurl]
classes = ['Bacterial Spot','Early Blight','Healthy','Late Blight','Septorial Leaf Spot','Yellow Curl Virus']
dfALL = CombineAllDF(folderPaths, classes)

In [ ]:

    
# Drop ID and LABELS
# Because this is to remove outliers only
cols = dfALL.drop(['ID','Labels'], axis = 1).columns

# Remove outliers
Q1 = dfALL[cols].quantile(0.25)
Q3 = dfALL[cols].quantile(0.75)
IQR = Q3 - Q1

dfFinal = dfALL[~((dfALL[cols] < (Q1 - 1.5 * IQR)) | (dfALL[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
dfFinal = dfFinal.reset_index()
dfFinal.drop('index',inplace = True, axis = 1)
dfFinal['Red3'] = dfFinal['Red3'].replace(np.nan, 0.277547)
dfFinal['Green3'] = dfFinal['Green3'].replace(np.nan, 0.34919)
dfFinal['Blue3'] = dfFinal['Blue3'].replace(np.nan, 0.340418)
print(dfFinal.shape)

# Generate X_train, without ID and Labels
# Y_train only contains labels
X = dfFinal.drop(['ID','Labels'], axis = 1)
y = dfFinal.loc[:,'Labels']

# split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
print('Training set shape: ', X_train.shape, y_train.shape)
print('Testing set shape: ', X_test.shape, y_test.shape)

In [ ]:

    
# List out all files that have been removed
dfOutliers = dfALL[((dfALL[cols] < (Q1 - 1.5 * IQR)) | (dfALL[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
for i in range(dfOutliers.shape[0]):
    print(dfOutliers['ID'].iloc[i])

## K-Fold Cross Validation

In [ ]:


# Use 10-fold cross validation
n_splits = 10

# Number of components/features after PCA
n_components = int(len(cols)/5)

kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)


# Create a list to store all metrics for training and evaluating
accuracies_tr = []
precisions_tr = []
recalls_tr = []
f1s_tr = []

accuracies_val = []
precisions_val = []
recalls_val = []
f1s_val = []


for i, (train_index, test_index) in enumerate(kf.split(X_train)):
    # split the training set into train and val sets
    X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, random_state = 42)

    
    # Mean Imputation
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(X_tr)
    imp.transform(X_tr)
    imp.transform(X_val)
    
    
    # Standard Scaler
    scaler = StandardScaler()
    scaler.fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_val = scaler.transform(X_val)
    
    
    # PCA
    pca = PCA(n_components = n_components)
    pca.fit(X_tr)
    X_tr = pca.transform(X_tr)
    X_val = pca.transform(X_val)
    
    
    # Classifier
    #clf =  #svm.SVC()
    clf = DecisionTreeClassifier(random_state = 42)
    clf.fit(X_tr, y_tr)
    y_pred_tr = clf.predict(X_tr)
    y_pred_val = clf.predict(X_val)
    
    
    # Print current split count
    print('Current Split: ', i)
    i+=1
    
    
    # Evaluate Train
    accuracy = accuracy_score(y_tr, y_pred_tr)
    pre, rec, f1, _ = precision_recall_fscore_support(y_tr, y_pred_tr, average='macro', zero_division = 0)
    print('Training')
    print('Accuracy: ',accuracy)
    print('Precision: ', pre)
    print('Recall: ', rec)
    print('F1: ', f1)
    con_tr = confusion_matrix(y_tr, y_pred_tr, labels=classes)#Labels

    
    accuracies_tr.append(accuracy)
    precisions_tr.append(pre)
    recalls_tr.append(rec)
    f1s_tr.append(f1)
    
    
    # Evaluate Val
    accuracy = accuracy_score(y_val, y_pred_val)
    pre, rec, f1, _ = precision_recall_fscore_support(y_val, y_pred_val, average='macro', zero_division = 0)
    print('\n')
    print('Validation')
    print('Accuracy: ',accuracy)
    print('Precision: ', pre)
    print('Recall: ', rec)
    print('F1: ', f1)
    con_val = confusion_matrix(y_val, y_pred_val, labels=classes)#Labels

    
    accuracies_val.append(accuracy)
    precisions_val.append(pre)
    recalls_val.append(rec)
    f1s_val.append(f1)
    
    
    print('\n')
    print('\n')
    print('\n')
    
print('Average training accuracy: ', mean(accuracies_tr))
print('Average training precision: ', mean(precisions_tr))
print('Average training recall: ', mean(recalls_tr))
print('Average training f1: ', mean(f1s_tr))
print('\n')
print('Average validation accuracy: ', mean(accuracies_val))
print('Average validation precision: ', mean(precisions_val))
print('Average validation recall: ', mean(recalls_val))
print('Average validation f1: ', mean(f1s_val))

## Hyperparameter Tuning

In [ ]:


# List out parameters to perform grid search
parameters = {'kernel':['linear', 'poly','rbf', 'sigmoid'], 
      'C':[0.1, 0.5, 1, 5, 10],
      'degree':[1, 3, 5, 7, 9],
      'class_weight':[None, 'balanced']
     }

# This one list all the scoring that we wants
scoring = ['accuracy','precision_macro','recall_macro','f1_macro']

# Transform input data
scaler = StandardScaler()
scaler.fit(X_train)
X_tr = scaler.transform(X_train)

pca = PCA(n_components = n_components)
pca.fit(X_tr)
X_tr = pca.transform(X_tr)

# Start hyperparameter tuning
clf = svm.SVC(decision_function_shape = 'ovr')
clf = GridSearchCV(clf, parameters, cv = 5, scoring = scoring, refit = 'accuracy')

# Fit the model on our train set
clf.fit(X_tr, y_train)

# Find the best score
print(clf.best_score_)

# Get the hyperparameters with the best score
print(clf.best_params_)
best_clf = clf.best_params_

# Record the best estimator
print(clf.best_estimator_)


### THERE MIGHT BE WARNING FOR PRECISION ILL-DEFINED DUE TO DIVISION BY 0